<a href="https://colab.research.google.com/github/DarkEol/AutoML-Ontology/blob/main/AutoH2O_Meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##script for extraction of meta-features from datasets along with information about algorithms selected by H2O AutoML

In [ ]:
!pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 MB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.40.0.4-py2.py3-none-any.whl size=177697886 sha256=bcc015de2f5e89893db9380d9bdeb181f495e1b2ff2e72de0632a289de82dc4a
  Stored in directory: /root/.cache/pip/wheels/43/f2/b0/5bb4d702a0467e82d77c45088db3eef25114c26b0eec8e7f6a
Successfully built h2o


In [ ]:
import h2o
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from h2o.automl import H2OAutoML
import sklearn.metrics

In [ ]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,5 hours 30 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,1 month and 3 days
H2O_cluster_name:,H2O_from_python_unknownUser_d2wz2m
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.060 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
mfeatures = pd.DataFrame(columns=["Name", "NumberOfInstances", "NumberOfFeatures", "ManyInstances", "FewInstances", "ManyFeatures", "FewFeatures", "BinaryClass",	"DateClass", "MissingClassValues", "NominalClass", "NumericClass", "NoClass",	"UnaryClass", "EmptyNominalClass", "StringClass", "RelationalClass", "OnlyMulti-InstanceData", "BinaryAttributes", "DateAttributes", "EmptyNominalAttributes", "MissingValues", "NominalAttributes", "NumericAttributes", "UnaryAttibutes",	"RelationalAttributes",	"StringAttributes", "Algorithm", "Accuracy"])
empty_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [ ]:
#datasets_nums = [6, 11, 14, 16, 18, 20, 22, 15, 23, 24, 28, 29, 32, 37, 38, 42, 44, 46, 50, 54, 60]
datasets_nums = [28]
i=0

for dset_num in datasets_nums:
  #binary_attributes = False
  #numeric_attributes = False
  #print(dset_num)

  opml = fetch_openml(data_id=dset_num, as_frame=True, parser='auto')
  data = opml.data

  #preparing data for H2O
  x=opml.feature_names
  y=opml.target.name
  frame = h2o.H2OFrame(opml.frame)
  train, test = frame.split_frame(ratios=[.75])

  name = opml.details['name']
  target = opml.target
  shape = data.shape
  n_instances = shape[0]
  n_features = shape[1]
  n_target_values = len(target.unique())
  print("name:", name)

  #print("instances:", n_instances)
  #print("features:", n_features)
  #print("target:", type(target))
  #print("target:", dir(target))
  #print("target:", n_target_values)

  numeric_data = data.select_dtypes(include=[np.number])
  #print("num_data:",numeric_data.shape[1])
  not_numeric_data = data.select_dtypes(exclude=[np.number])
  #print("not_num_data:",not_numeric_data.shape[1])
  category_data = data.select_dtypes(include=['category'])
  #print("category_data:",category_data.shape[1])
  object_data = data.select_dtypes(include=[object])
  #print("object_data:",object_data.shape[1])

  #print("target:", target.astype("str"))

  #print("target:", target)
  #print("target:", target.str.isnumeric())

#mfeatures = pd.DataFrame([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]],columns=["Name", "NumberOfInstances", "NumberOfFeatures", "BinaryClass",	"DateClass", "MissingClassValues", "NominalClass", "NumericClass", "NoClass",	"UnaryClass", "EmptyNominalClass", "StringClass", "RelationalClass", "OnlyMulti-InstanceData", "BinaryAttributes", "DateAttributes", "EmptyNominalAttributes", "MissingValues", "NominalAttributes", "NumericAttributes", "UnaryAttibutes",	"RelationalAttributes",	"StringAttributes"])

  mfeatures.loc[len(mfeatures)] = empty_list #adding row 

  mfeatures.at[i,'Name'] = name
  mfeatures.at[i,'NumberOfInstances'] = n_instances
  mfeatures.at[i,'NumberOfFeatures'] = n_features
  
  if n_instances > 3200:
    mfeatures.at[i,'ManyInstances'] = 1
  else:
    mfeatures.at[i,'FewInstances'] = 1
  
  if n_features > 100:
    mfeatures.at[i,'ManyFeatures'] = 1
  else:
    mfeatures.at[i, 'FewFeatures'] = 1

  if n_target_values == 2:
    mfeatures.at[i, 'BinaryClass'] = 1

  #if target.str.isnumeric().any():
  #if pd.api.types.is_numeric_dtype(target):
  #  mfeatures.at[i, 'NumericClass'] = 1
  #else:
  #  if pd.api.types.is_string_dtype(target):
  #    mfeatures.at[i, 'StringClass'] = 1
  #  else:
  #    mfeatures.at[i, 'NominalClass'] = 1

  if target.str.isnumeric().any():
    mfeatures.at[i, 'NumericClass'] = 1
  else:
    mfeatures.at[i, 'StringClass'] = 1

  if target.dtype=='category':
    mfeatures.at[i, 'NominalClass'] = 1

  if numeric_data.shape[1] > 0:
    mfeatures.at[i, 'NumericAttributes'] = 1
  
  if category_data.shape[1] > 0:
    mfeatures.at[i, 'NominalAttributes'] = 1

  for fname in opml.feature_names:
    #print("type",data[fname].str.isnumeric())
    #print(len(data[fname].unique()))
    if len(data[fname].unique())==2: # and not binary_attributes:
      mfeatures.at[i, 'BinaryAttributes'] = 1

    if len(data[fname].unique())==1:
      mfeatures.at[i, 'UnaryAttibutes'] = 1

      #binary_attributes = True
    if data[fname].isna().sum() > 0:
      mfeatures.at[i, 'MissingValues'] = 1

    if not pd.api.types.is_numeric_dtype(data[fname]):
      mfeatures.at[i, 'StringAttributes'] = 1

  #preparing autoML
  if train[y].isnumeric():
    train[y] = train[y].asfactor()
    test[y] = test[y].asfactor()
    print("convert numeric target to categorial")
  else:
    print("target remained the same")

  automl = h2o.automl.H2OAutoML(max_runtime_secs=600)

  print("searching for model...")
  automl.train(x=x, y=y, training_frame=train)

  algo = automl.leader.key
  if n_target_values == 2: #metrics for binary classifications
    perf = automl.leader.model_performance(test)
    accuracy = perf.accuracy()[0][1]
  else:
    y_test = test[y].as_data_frame() 
    y_pred_0 = automl.leader.predict(test)  #calculate predictions
    y_pred = y_pred_0['predict'].as_data_frame() #take from table only predictions
    accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)

  print("algo: ",algo)
  print("score: ",accuracy)
  mfeatures.at[i, 'Algorithm'] = algo
  mfeatures.at[i, 'Accuracy'] = accuracy

    #if pd.api.types.is_numeric_dtype(data[fname]): # or data[fname].str.isnumeric().any():
    #  mfeatures.at[i, 'NumericAttributes'] = 1
    #else: 
    #  if pd.api.types.is_string_dtype(data[fname]):
    #    mfeatures.at[i, 'StringAttributes'] = 1 
    #  else:
    #    mfeatures.at[i, 'NominalAttributes'] = 1

  #if binary_attributes:
  #  mfeatures.at[i, 'BinaryAttributes'] = 1
  i+=1

print(mfeatures[['Name','ManyInstances', 'FewInstances', 'ManyFeatures', 'FewFeatures', 'BinaryClass','NumericClass','StringClass','NominalClass','BinaryAttributes','MissingValues','NominalAttributes','NumericAttributes','StringAttributes','UnaryAttibutes']])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
name: optdigits
convert numeric target to categorial
searching for model...
AutoML progress: |
21:44:44.713: _train param, Dropping bad and constant columns: [input1, input40]

██████
21:45:28.744: _train param, Dropping bad and constant columns: [input1, input40]

████
21:46:10.417: GLM_1_AutoML_22_20230531_214444 [GLM def_1] failed: java.lang.ArrayIndexOutOfBoundsException: Index 62 out of bounds for length 62
21:46:10.429: _train param, Dropping bad and constant columns: [input1, input40]

██████████████████████
21:49:47.334: _train param, Dropping unused columns: [input1, input40]

██
21:50:07.115: _train param, Dropping bad and constant columns: [input1, input40]

████
21:50:43.730: _train param, Dropping bad and constant columns: [input1, input40]

████
21:51:21.862: _train param, Dropping bad and constant columns: [input1, input40]

████
21:52:04.772: _train param, Dropping bad and con

In [ ]:
print(mfeatures[['Name','ManyInstances', 'FewInstances', 'ManyFeatures', 'FewFeatures', 'BinaryClass','NumericClass','StringClass','NominalClass','BinaryAttributes','MissingValues','NominalAttributes','NumericAttributes','StringAttributes','UnaryAttibutes','Algorithm','Accuracy']])
print(mfeatures.shape)
mfeatures.to_csv('features.csv')
mfeatures.to_excel('features.xlsx')

                   Name  ManyInstances  FewInstances  ManyFeatures  \
0                letter              1             0             0   
1         balance-scale              0             1             0   
2         mfeat-fourier              0             1             0   
3        mfeat-karhunen              0             1             0   
4   mfeat-morphological              0             1             0   
5           mfeat-pixel              0             1             1   
6         mfeat-zernike              0             1             0   
7              breast-w              0             1             0   
8                   cmc              0             1             0   
9              mushroom              1             0             0   
10            optdigits              1             0             0   
11      credit-approval              0             1             0   

    FewFeatures  BinaryClass  NumericClass  StringClass  NominalClass  \
0             1 